# Cohort 05 

In [2]:
import requests
import datetime
import pandas as pd

from evidently import ColumnMapping
from evidently.report import Report
from evidently.metrics import ColumnDriftMetric, DatasetDriftMetric, DatasetMissingValuesMetric

from joblib import load, dump
from tqdm import tqdm

from sklearn.linear_model import LinearRegression

C:\Users\USUARIO\.conda\envs\py11\Lib\site-packages\umap\distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
C:\Users\USUARIO\.conda\envs\py11\Lib\site-packages\umap\distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
C:\Users\USUARIO\.conda\envs\py11\Lib\site-packages\umap\distances.py:1086: NumbaDeprecationWa

### Q01 Prepare the dataset

In [3]:
files = [('green_tripdata_2022-02.parquet', './data'), ('green_tripdata_2022-01.parquet', './data'), ('green_tripdata_2022-03.parquet', './data')]

print("Download files:")
for file, path in files:
    url=f"https://d37ci6vzurychx.cloudfront.net/trip-data/{file}"
    resp=requests.get(url, stream=True)
    save_path=f"{path}/{file}"
    with open(save_path, "wb") as handle:
        for data in tqdm(resp.iter_content(),
                        desc=f"{file}",
                        postfix=f"save to {save_path}",
                        total=int(resp.headers["Content-Length"])):
            handle.write(data)

Download files:


green_tripdata_2022-03.parquet: 100%|████████████████| 1615562/1615562 [00:20<00:00, 77382.67it/s, save to ./data/green_tripdata_2022-03.parquet]


In [4]:
jan_data = pd.read_parquet('data/green_tripdata_2022-01.parquet')
mar_data = pd.read_parquet('data/green_tripdata_2022-03.parquet')

**What is the shape of the downloaded data? How many rows are there?**

In [5]:
mar_data.shape

(78537, 20)

In [6]:
#create target
jan_data['duration_min']= jan_data.lpep_dropoff_datetime-jan_data.lpep_pickup_datetime
jan_data.duration_min = jan_data.duration_min.apply(lambda td: float(td.total_seconds())/60)
# filter out outliers
jan_data = jan_data[(jan_data.duration_min >= 0) & (jan_data.duration_min <= 60)]
jan_data = jan_data[(jan_data.passenger_count > 0) & (jan_data.passenger_count <= 8)]

In [7]:
#data labeling
target = 'duration_min'
num_features = ['passenger_count', 'trip_distance', 'fare_amount', 'total_amount']
cat_features = ['PULocationID', 'DOLocationID']

In [8]:
train_data = jan_data[:30000]
val_data = jan_data[30000:]

In [9]:
model = LinearRegression()
model.fit(train_data[num_features + cat_features], train_data[target])

LinearRegression()

In [11]:
with open('models/lin_reg.bin', 'wb') as f_out:
    dump(model, f_out)

In [12]:
val_data.to_parquet('data/reference.parquet')

### Q02 Metric

In [13]:
column_mapping = ColumnMapping(
    target= None,
    prediction= 'prediction',
    numerical_features= num_features,
    categorical_features= cat_features
    )

In [14]:
from evidently.metrics import ColumnQuantileMetric

In [15]:
report = Report(metrics= [
    ColumnDriftMetric(column_name= 'prediction'),
    DatasetDriftMetric(),
    DatasetMissingValuesMetric(),
    ColumnQuantileMetric(column_name="fare_amount", quantile=0.50)
    ]
    )

In [16]:
report.run(reference_data= train_data, current_data= val_data, column_mapping= column_mapping)

In [17]:
results = report.as_dict()
results['metrics'][3]

{'metric': 'ColumnQuantileMetric',
 'result': {'column_name': 'fare_amount',
  'column_type': 'num',
  'quantile': 0.5,
  'current': {'value': 10.0},
  'reference': {'value': 10.0}}}

### Q04 Monitoring

In [19]:
import psycopg

In [31]:
conn = psycopg.connect("host=localhost port=5432 dbname=cohort05 user=postgres password=cohort05", autocommit=True)

**What is the maximum value of metric quantile = 0.5 on th "fare_amount" column during March 2023 (calculated daily)?**

In [46]:
res = conn.execute('''
    SELECT max(columns_quantile)
    FROM dummy_metrics
    ''')
res.fetchone()

(11.5,)